In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import copy
import random
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
import random

In [ ]:
df_unbalanced = pd.read_csv("Sampled_balanced.csv")

In [ ]:
df_unbalanced

,Unnamed: 0,index,ID,Text,Domain,URL,Content,Match
0,0,0,1.324130e+18,“For Premier Doug Ford to say that they've don...,Medical,https://t.co/vk89LoQfZp,And it's not just Ontario. Long lines and shor...,1
1,1,1,1.324130e+18,"In any season, getting sick is tough. But rig...",Medical,https://t.co/OsHSfvef52,AboutPressCopyrightContact usCreatorsAdvertise...,1
2,2,2,1.324130e+18,@FatEmperor And the lastest vicim of the swine...,Medical,https://t.co/HPkErToAHc,"To date, more than 80 cases have been lodged b...",1
3,3,3,1.324130e+18,Fargo Cass Public Health holds additional flu ...,Medical,https://t.co/ztEuaglz07,FARGO — Due to the positive response to last w...,1
4,4,4,1.324130e+18,Kitchener man Hosherdar Polad says he feels di...,Medical,https://t.co/XGEhe89mhn,WATERLOO — Getting a flu shot seemed especiall...,1
...,...,...,...,...,...,...,...,...
23719,23719,1313,1.324440e+18,This Type of Meat Is Linked to Increased Cance...,unknown,https://t.co/2u2KRJSDj9,This Type of Meat Is Linked to Increased Cance...,1
23720,23720,1314,1.324440e+18,@BluebirdsofMO @collhistgarden @HawleyMO @GOP ...,unknown,https://t.co/4vbi3brgeq,Missouri Attorney General Josh Hawley doubled ...,1
23721,23721,1315,1.324440e+18,Quite a mystery!,unknown,https://t.co/vycImaqSZ3,OU Medicine expert looks at why Oklahoma's see...,1
23722,23722,1316,1.324440e+18,I wrote this back in March along with some oth...,unknown,https://t.co/UgrY57yXvP,At the moment I think it’s fair to say that ma...,1


In [ ]:
#df_balanced = pd.read_csv("Sampled_balanced.csv")

In [ ]:
df_unbalanced['Match'].mean()

0.5

In [ ]:
vectorizer_tweet = TfidfVectorizer(max_df = 200,max_features=300)
vectorizer_content = TfidfVectorizer(max_df = 200,max_features=300)
tweet_tfidf = vectorizer_tweet.fit_transform(df_unbalanced["Text"])
content_tfidf = vectorizer_content.fit_transform(df_unbalanced["Content"])

In [ ]:
import scipy.sparse as sp

In [ ]:
A_tweet_tfidf = tweet_tfidf.A
A_content_tfidf = content_tfidf.A

In [ ]:
concat_tfidf =np.hstack((A_tweet_tfidf, A_content_tfidf))

In [ ]:
del A_content_tfidf
del A_tweet_tfidf

In [ ]:
y = df_unbalanced["Match"]

In [ ]:
X_train, X_test, y_train, y_test =  train_test_split(concat_tfidf, y, test_size=0.3, random_state=42,stratify = y)

In [ ]:
X_train.shape

(16606, 600)

Logistic Regreesion

In [ ]:
clf1 = LogisticRegression(class_weight='balanced').fit(X_train, y_train)

In [ ]:
y_pred1 =clf1.predict(X_test)
clf1.score(X_test,y_test)

0.4442259061534139

In [ ]:
f1_score(y_test, y_pred1)

0.3995142683667274

In [ ]:
for i in [10,20,30,50,100]:
    
    model1 = LogisticRegression(class_weight={1: i,0:1}, max_iter=10000)
    model1.fit(X_train, y_train)
    
    y_pred=model1.predict(X_test)
    model1.score(X_test,y_test)
    print(f1_score(y_test, y_pred))

    print('recall',recall_score(y_test, y_pred))

0.6666666666666666


C:\Users\matia\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


NameError: name 'recall_score' is not defined

Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model2 = DecisionTreeClassifier(class_weight='balanced')
model2.fit(X_train, y_train)

In [ ]:
y_pred=model2.predict(X_test)
model2.score(X_test,y_test)

In [ ]:
f1_score(y_test, y_pred)

In [ ]:
for i in [10,20,30,50,100]:
    
    model2 = DecisionTreeClassifier(class_weight={1: i,0:1})
    model2.fit(X_train, y_train)
    
    y_pred=model2.predict(X_test)
    model2.score(X_test,y_test)
    print(f1_score(y_test, y_pred))

    print('recall',recall_score(y_test, y_pred))

Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
param_grid = [{'max_depth':[2,4,8,10]}]
kfold = StratifiedKFold(n_splits=10, shuffle = True)
grid = GridSearchCV(RandomForestClassifier(), param_grid, n_jobs=4, cv = kfold, scoring = 'accuracy')

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
grid_values = list(grid.best_params_.values())

In [ ]:
grid_values

In [ ]:
model3 = RandomForestClassifier(n_estimators = 500, max_depth=2, class_weight='balanced')
model3.fit(X_train, y_train)

In [ ]:
y_pred=model3.predict(X_test)
model3.score(X_test,y_test)

In [ ]:
f1_score(y_test, y_pred)

In [ ]:
from sklearn.metrics import recall_score
s = np.random.binomial(1,0.5, len(y_pred))
print(f1_score(y_test, s))

print('recall',recall_score(y_test, s))

Re-sampling the train set

In [ ]:
y_train.value_counts()[1]

In [ ]:
X_train_rs = X_train
y_train_rs = y_train

In [ ]:
diff = y_train.value_counts()[0] - y_train.value_counts()[1]
diff

In [ ]:
ones = X_train_rs[y_train_rs == 1]

In [ ]:
for i in range(0,diff):
  temp = random.choice(ones)
  X_train_rs = np.vstack([X_train_rs, temp])
  y_train_rs = np.append(y_train_rs,1)

In [ ]:
X_train_rs.shape

In [ ]:
np.bincount(y_train_rs)

randomize = np.arange(len(X_train_rs))
np.random.shuffle(randomize)
X_train_rs_r = X_train_rs[randomize]
y_train_rs_r = y_train_rs[randomize]

X_train_rs_r.shape

Naive Bayes

In [ ]:
from sklearn.naive_bayes import BernoulliNB

In [ ]:
model4 = BernoulliNB()
model4.fit(X_train, y_train)

In [ ]:
y_pred = model4.predict(X_test)
model4.score(X_test,y_test)

In [ ]:
f1_score(y_test, y_pred)

MLP model


In [ ]:
from sklearn.neural_network import MLPClassifier
model5 = MLPClassifier()
model5.fit(X_train, y_train)

In [ ]:
y_pred = model5.predict(X_test)
model5.score(X_test,y_test)

In [ ]:
print(f1_score(y_test, y_pred))
print('recall',recall_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import recall_score
s = np.random.binomial(1, y_pred.mean(), len(y_pred))
print(f1_score(y_test, s))

print('recall',recall_score(y_test, s))

In [ ]:
ppt_str = df_unbalanced["Text"][0]

In [ ]:
df_unbalanced[df_unbalanced["Text"] == ppt_str]